In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, NoSuchWindowException
from selectorlib import Extractor
import bs4
import requests
import json
import time
import urllib.request
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import csv
import pandas as pd
import numpy as np
from datetime import date
import py_files.fbref_scraper as fbref_scraper
# standard_stats_scraper, shooting_stats_scraper, passing_stats_scraper, gsc_stats_scraper, 
# defense_stats_scraper, possession_stats_scraper,pass_type_stats_scraper, miscellaneous_stats_scraper, 
# gk_stats_scraper

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/ericroberts/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


In [ ]:
url = "https://fbref.com/en/comps/20/3248/stats/2019-2020-Bundesliga-Stats"
url = "https://fbref.com/en/comps/20/2109/stats/2018-2019-Bundesliga-Stats"
      "https://fbref.com/en/comps/9/3232/stats/2019-2020-Premier-League-Stats"
      "https://fbref.com/en/comps/9/1889/stats/2018-2019-Premier-League-Stats"
      "https://fbref.com/en/comps/9/1889/passing_types/2018-2019-Premier-League-Stats"
      
url_passing = "https://fbref.com/en/comps/20/3248/passing/2019-2020-Bundesliga-Stats"

generic_url = f'"https://fbref.com/en/comps/{leaguenum}/{yearnum?}/{stat_table}/{year}-{league}-Stats"

In [9]:
league = ['Bundesliga','Premier-League','Ligue-1','Serie-A','La-Liga']
year = ['2017-2018','2018-2019','2019-2020']
leaguenum = ['20','9','13','11','12']
###WHY DO THEY HAVE A NUMBER IN THE URL THAT IS UNIQUE TO LEAGUE AND NUMBER AND NOT REGULARLY SPACED???
#SOLVE BY CREATING A DICTIONARY TO READ - HAVE TO GET THIS NUMBER FROM URL OF CORRESPONDING LEAGUE AND YEAR
league_year_num = {'Bundesliga_2017-2018':'1634','Bundesliga_2018-2019':'2109','Bundesliga_2019-2020':'3248',
                   'Premier-League_2017-2018':'1631','Premier-League_2018-2019':'1889','Premier-League_2019-2020':'3232',
                  'Ligue-1_2017-2018':'1632','Ligue-1_2018-2019':'2104','Ligue-1_2019-2020':'3243',
                  'Serie-A_2017-2018':'1640','Serie-A_2018-2019':'1896','Serie-A_2019-2020':'3260',
                  'La-Liga_2017-2018':'1652','La-Liga_2018-2019':'1886','La-Liga_2019-2020':'3239'}

# stat_table = ['stats', 'shooting', 'passing', 'gca', 'defense', 'possession', 'passing_types', 'misc', 'keepers']

In [10]:
all_leagues_all_years = pd.DataFrame()
for i in range(len(league)):
    #loop through league and leaguenum using i
    all_years = pd.DataFrame()
    for j in range(len(year)):
        #loop through year and yearnum using j
        all_stats = pd.DataFrame()
        
        key = league[i]+"_"+year[j]
        league_year = league_year_num[key]
        
        url = "https://fbref.com/en/comps/"+leaguenum[i]+"/"+league_year+"/stats/"+year[j]+"-"+league[i]+"-Stats"
        df_standard = fbref_scraper.standard_stats_scraper(url)
        
        url = "https://fbref.com/en/comps/"+leaguenum[i]+"/"+league_year+"/shooting/"+year[j]+"-"+league[i]+"-Stats"
        df_shooting = fbref_scraper.shooting_stats_scraper(url)
        
        url = "https://fbref.com/en/comps/"+leaguenum[i]+"/"+league_year+"/passing/"+year[j]+"-"+league[i]+"-Stats"
        df_passing = fbref_scraper.passing_stats_scraper(url)
        
        url = "https://fbref.com/en/comps/"+leaguenum[i]+"/"+league_year+"/gca/"+year[j]+"-"+league[i]+"-Stats"
        df_gsc = fbref_scraper.gsc_stats_scraper(url)            
        
        url = "https://fbref.com/en/comps/"+leaguenum[i]+"/"+league_year+"/defense/"+year[j]+"-"+league[i]+"-Stats"
        df_defense = fbref_scraper.defense_stats_scraper(url)
        
        url = "https://fbref.com/en/comps/"+leaguenum[i]+"/"+league_year+"/possession/"+year[j]+"-"+league[i]+"-Stats"
        df_possession = fbref_scraper.possession_stats_scraper(url)
        
        url = "https://fbref.com/en/comps/"+leaguenum[i]+"/"+league_year+"/passing_types/"+year[j]+"-"+league[i]+"-Stats"
        df_pass_type = fbref_scraper.pass_type_stats_scraper(url)
        
        url = "https://fbref.com/en/comps/"+leaguenum[i]+"/"+league_year+"/misc/"+year[j]+"-"+league[i]+"-Stats"
        df_miscellaneous = fbref_scraper.miscellaneous_stats_scraper(url)
        
        url = "https://fbref.com/en/comps/"+leaguenum[i]+"/"+league_year+"/keepers/"+year[j]+"-"+league[i]+"-Stats"
        df_gk = fbref_scraper.gk_stats_scraper(url)
        
        all_stats = pd.concat([df_standard, df_shooting,df_passing,df_gsc,df_defense,df_possession,df_pass_type,df_miscellaneous], axis=1)
        all_stats['year'] = year[j]
        all_years = all_years.append(all_stats)
    all_years['league'] = league[i]
    all_leagues_all_years = all_leagues_all_years.append(all_years)

In [11]:
all_leagues_all_years

,,,,,,games,games_start,mins,goals,assists,pens_successful,pens_attempts,yellow_cards,red_cards,goals_per_90,...,passes_received_pct,miscontrols,dispossessed,passes_left_foot,passes_right_foot,aerials_won,aerials_lost,aerials_won_pct,year,league
players,nationality,team,position,age,birth_year,,,,,,,,,,,,,,,,,,,,,
David Abraham,ar ARG,Eint Frankfurt,DF,31,1986,27,27,"2,302",0,2,0,0,3,0,0.00,...,98.6,5,6,208,923,62,26,70.5,2017-2018,Bundesliga
Amir Abrashi,al ALB,Freiburg,MF,27,1990,12,11,850,0,0,0,0,2,0,0.00,...,86.5,9,14,18,272,21,19,52.5,2017-2018,Bundesliga
René Adler,de GER,Mainz 05,GK,32,1985,14,14,"1,260",0,0,0,0,0,0,0.00,...,100.0,0,0,39,284,0,1,0.0,2017-2018,Bundesliga
Ailton,br BRA,Stuttgart,"DF,FW",22,1995,5,1,108,0,0,0,0,0,0,0.00,...,86.8,3,0,46,3,2,3,40.0,2017-2018,Bundesliga
Manuel Akanji,ch SUI,Dortmund,DF,22,1995,11,10,904,0,0,0,0,2,0,0.00,...,98.1,5,3,142,394,17,17,50.0,2017-2018,Bundesliga
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Óscar Plano,es ESP,Valladolid,MF,28,1991,36,28,"2,505",4,2,0,0,6,0,0.14,...,74.0,44,26,546,105,26,64,28.9,2019-2020,La-Liga
Óscar Rodríguez Arnaiz,es ESP,Leganés,"MF,FW",21,1998,30,22,"1,914",9,2,2,2,6,1,0.42,...,78.8,42,31,103,650,15,36,29.4,2019-2020,La-Liga
Óscar de Marcos,es ESP,Athletic Club,"DF,FW",30,1989,13,6,541,0,1,0,0,1,0,0.00,...,75.6,6,3,14,166,7,13,35.0,2019-2020,La-Liga


In [12]:
all_leagues_all_years.to_csv('data/Player_stats_top_5_leagues_2017_to_2020.csv')